# Browse and Explore ASAP CRN Curated Data (python)

# Part 3: Gene of interest analysis (__Workshop__)

## Overview

This notebook shows how to load ASAP CRN Cloud data for the harmonized cohort of scRNAseq datasets.  (All contrubted datasets in this collection are technically single-_nucleus_ samples.)   

Part 1: ASAP CRN Cloud __Basics__
* Browse ASAP CRN curated datasets and collections on a cloud environment
* Define paths to dataset 
    * cell-wise data metrics (_cell_ metadata)
    * gene expression data (anndata objects)
    * _dataset_ metadata
* Load _cell_-metadata and visualize
    * basic QC visualizations
    * UMAP visualizations
* Load _processed_ `AnnData` object 
    * access gene expression 
    * UMAP visualizations
    * Load _dataset_ metadata
        * merge _dataset_ metadata with _cell_ metadata
        * create Case-Control and Brain Region sub-sets


Part 2: Create subset `AnnData` artifacts (__Intermediate__)
* Load unfiltered dataset `AnnData` object
* combine with _processed_ artifacts (`adata.obs`) 
* Subset to a single brain region and Case-Contol group (from _dataset_ metadata)
    * save full gene expression `anndata` for subset with full metadata annotation (_dataset_ and _cell_-level)

Part 3: Gene of interest analysis (__Workshop__)
* Choose Parkinson's Disease relavent gene
* Load transciptomics counts data into an anndata object
    * subset cells to subset samples from metadata
* visualize specific gene expression

    
Part 4: Differential Expression (DE) Analysis (__Advanced__)
* Differential expression analysis,
    * psuedobulk over 'sample' and 'cell_type' 
    * logistic regression CASE vs. CONTROL
    * meta analysis to verify consistency across Datasets / Teams
* Differential expression analysis meta-analysis
    * psuedobulk over 'sample' and 'cell_type' 
    * logistic regression CASE vs. CONTROL per source dataset
    * meta analysis to verify consistency across Datasets / Teams


------------------------------
>> NOTES
>>   need to test that this runs reasonably fast on a generic VM
>>   need to develop an equivalent R version

## Imports

###  ASAP CRN data paths
First, let's build the paths to our data. 

We'll focus in on the datasets processed with our *PMDBS scRNAseq* workflow.  Specifically the _cohort_ dataset: `asap-cohort-pmdbs-sc-rnaseq`.  This dataset includes samples from 5 Contributing datasets which have been processesed and integrated.  The paths include the following parts.

- `workflow` designates the workflow which performs the aggregation and integration.  In this case the [*PMDBS scRNAseq* workflow](https://github.com/ASAP-CRN/pmdbs-sc-rnaseq-wf)
- `dataset_team` designates the contributing team for the dataset.  In this case _cohort_ designates that it is made from multiple individual contributed datasets.
- `source` designates the _source_ of the samples.  In this case Post-mortem derived Brain samples
- `dataset_type` designates the 
- `bucket_name` designates the datasets gcp bucket
- `dataset_name` designates the unique designation for each dataset or collection

#### Cohort Analysis Path

Now that we've defined the path to our cohort dataset, lets list the curated files for the `cohort_analysis`.   

#### Dataset Metadata Path

The dataset metadata can be found in the `release_resources`.  Note that the metadata are organized by the _short_ `dataset_name` rather than `bucket_name`.  



#### Workshop/Notebook Artifact Path

This is the path to where the artifacts we've created in these notebooks can be found in case you want to skip ahead.  Particularly for the workshop.

#### Local Data Path
Lets also define a path for copying our data files and exporting intermediate analysis artifacts to your workspace.  In this example we'll make a "workshop_files" in the "ws_files" which are persistent in our Verily Workbench Workspace. 

-----------------
#  Part 3: Gene of interest analysis (__Workshop__)
* Gene of interest analysis
    * Choose Parkinson's Disease relavent gene
    * Load transciptomics counts data into an anndata object
        * subset cells to subset samples from metadata
    * visualize specific gene expression

## choose your favorite genes
Now that we have a 
1. check that your gene of interest is in the current features
2. color the umap by the expression level of your gene.


We'll use the soon-to-be-published table from GP2 as a reference set of putative PD variable genes.



### Load Parkinson's reference genes.

### Load full frontal cortex `AnnData` object for further analysis 

If you calculated these yourself in [Part 2](add link to notebook) load if from where you saved it. (e.g. 'local_data_path').  It is also made it available in the `WORKSHOP_PATH`.


Check that we have the genes of interest.

### Choose 'query_gene' and visualize

As an example below we'll query "TSC22D4".  What gene are you interested in?

--------------------
# Provenance
Generate information about this notebook environment and the packages installed.

Conda and pip installed packages:

JupyterLab extensions:

Number of cores:

Memory:

In [ ]:
# R doesn't need numpy, use base functions
library(dplyr)
library(data.table)

# Use pathlib for file path manipulation
library(fs) # File system operations 


# matplotlib and seaborn a pythonic alternative to plotnine
try:
    library(ggplot2)
    # Use ggplot2 instead of matplotlib
except ImportError as e:
    print("Error -> ", e)
    print("Installing seaborn or matplotlib")
    # Install manually: install.packages() matplotlib seaborn
    library(ggplot2)
    # Use ggplot2 instead of matplotlib


try:
    # scanpy has no direct R equivalent, consider Seurat
except ImportError as e:
    print("Error -> ", e)
    print("Installing scanpy")
    # Install manually: install.packages() scanpy
    # scanpy has no direct R equivalent, consider Seurat

try:
    %load_ext autotime
except:
    # Install manually: install.packages() ipython-autotime
    %load_ext autotime

# Always show all columns in a Pandas DataFrame
options(dplyr.width = Inf)

In [ ]:
# Workspace Path
HOME_PATH = path_home()
WS_PATH =  HOME_PATH / 'workspace'
if not WS_PATH.exists():
    print(f"{WS_PATH} doesn't exist. We need to remount our resources")
    !terra resource mount    

In [ ]:
DATASETS_PATH = WS_PATH / "01_PMDBS_scRNAseq_Datasets"
workflow = "pmdbs_sc_rnaseq"
dataset_team = "cohort"
dataset_source = "pmdbs"
dataset_type = "sc-rnaseq"
bucket_name = f"asap-curated-{dataset_team}-{dataset_source}-{dataset_type}"
dataset_name = f"asap-{dataset_team}-{dataset_source}-{dataset_type}"
dataset_path = DATASETS_PATH / bucket_name / workflow

In [ ]:
cohort_analysis_path = dataset_path / "cohort_analysis"

In [ ]:
ds_metadata_path = WS_PATH / 'release_resources/asap-crn-cloud-release-resources' / dataset_name / "metadata"
system("ls") {ds_metadata_path} # pythonic way: #[f.name for f in ds_metadata_path.glob("**/*.csv") if f.is_file()]


In [ ]:
WORKSHOP_PATH = WS_PATH / 'release_resources/asap-crn-cloud-release-resources/release-artifacts/sample_notebooks/workshop' 

system("ls") {WORKSHOP_PATH}


In [ ]:
local_data_path = WS_PATH / "ws_files" / "workshop_files"

if not local_data_path.exists():
    local_data_path.mkdir(parents=True)

In [ ]:
WORKSHOP_PATH =  WS_PATH / 'release_resources/asap-crn-cloud-release-resources/release-artifacts/sample_notebooks/workshop' 
ref_gene_filename = WORKSHOP_PATH / 'GP2 European GWAS Manuscript Tables - Table 4.csv'

GP2_PD_genes = pd.read_csv(ref_gene_filename)

In [ ]:
full_frontal_samples_filename = (
    WORKSHOP_PATH / f"asap-{dataset_team}.full_frontal_ctx_case_control_samples.h5ad"
)
frontal_full_ad = sc.read_h5ad(full_frontal_samples_filename)

In [ ]:
pd_genes = set(GP2_PD_genes["SYMBOL"]).intersection(
    set(frontal_full_ad.var_names)
)

In [ ]:
pd_genes

In [ ]:
query_gene = "CTSB"
query_gene = "TSC22D4"


CTSB_expr = frontal_full_ad[:, query_gene].X.toarray()
cell_metadata_df = pd.DataFrame(
    CTSB_expr, columns=[query_gene], index=frontal_full_ad.obs.index
)

cell_metadata_df.loc[:, "cell_type"] = frontal_full_ad.obs["cell_type"]

cell_metadata_df.groupby("cell_type").describe()

In [ ]:


sc.pl.embedding(
    frontal_full_ad,
    basis="umap",
    color=[query_gene],
    cmap=sns.cubehelix_palette(dark=0, light=0.9, as_cmap=True),
    title=f"{query_gene} gexp",
)

In [ ]:
# maybe calculate the relative expression 

In [ ]:
!date

In [ ]:
!pip freeze

In [ ]:
!jupyter labextension list

In [ ]:
!grep ^processor /proc/cpuinfo | wc -l

In [ ]:
!grep "^MemTotal:" /proc/meminfo